In [39]:
import time
from selenium import webdriver
from pprint import pprint
from bs4 import BeautifulSoup
import re

# Barça vs Real: https://onefootball.com/pt-br/match/2269972

driver = webdriver.Chrome('chromedriver.exe')  # Optional argument, if not specified will search path.
driver.get('https://onefootball.com/pt-br/match/2269971');

C:\Users\user\AppData\Local\Temp\ipykernel_15652\1043150601.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')  # Optional argument, if not specified will search path.


In [40]:
def get_stats(list_of_elements):
    stats_list = []
    for stat in list_of_elements:
        stats_text = stat.text.split('\n')
        home_team_stat = stats_text[0]
        stat_name = stats_text[1]
        away_team_stat = stats_text[2]
        stats_detail = {
            'home_team_stat': home_team_stat,
            'away_team_stat': away_team_stat,
            'stat_name': stat_name
        }
        stats_list.append(stats_detail)
    return stats_list

## Aceitar os cookies

In [41]:
search_box = driver.find_elements_by_xpath("//button[contains(@aria-label, 'Agree to our data processing and close')]")
search_box[0].click()

C:\Users\user\AppData\Local\Temp\ipykernel_15652\4220912207.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@aria-label, 'Agree to our data processing and close')]")


## Quem está jogando

In [5]:
search = driver.find_elements_by_xpath("//of-match-score-team")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\4027852203.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//of-match-score-team")


In [6]:
search[0].text

'Wolfsburgo Feminino'

In [7]:
def get_teams(list_of_elements):
    teams = {
        'home_team': list_of_elements[0].text,
        'away_team': list_of_elements[1].text,
    }
    return teams

In [8]:
teams = get_teams(search)

In [9]:
pprint(teams)

{'away_team': 'Arsenal Feminino', 'home_team': 'Wolfsburgo Feminino'}


## Placar

In [10]:
search = driver.find_elements_by_xpath("//p[contains(@class, 'match-score-scores')]")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\335478207.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//p[contains(@class, 'match-score-scores')]")


In [11]:
search[0].text

'2\n:\n0'

In [12]:
def get_score(list_of_elements):
    score_element = list_of_elements[0].text.split('\n')
    scores = {
        'home_team_score': score_element[0],
        'away_team_score': score_element[2]
    }
    return scores

In [13]:
score = get_score(search)

In [14]:
pprint(score)

{'away_team_score': '0', 'home_team_score': '2'}


## Escalações

In [15]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [16]:
for tag in soup.findAll('span', attrs={'class': re.compile('match-lineup-v2__player-name')}):
    print(tag.text)

 14. Roord 
 28. Waßmuth 
 32. Jónsdóttir 
 8. Lattwein 
 5. Oberdorf 
 10. Huth 
 13. Rauch 
 6. Janssen 
 4. Hendrich 
 2. Wilms 
 1. Schult 


In [17]:
search_box = driver.find_elements_by_xpath("//button[contains(@title, '" + teams['away_team'] + "')]")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\1664311966.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@title, '" + teams['away_team'] + "')]")


In [18]:
search_box

[<selenium.webdriver.remote.webelement.WebElement (session="be51f7a555746dca4cca1d5bbde571b2", element="eff70ad5-29a6-46a4-a032-976f1befe2d0")>]

In [19]:
search_box[0].click()

In [20]:
html = driver.page_source

In [21]:
soup = BeautifulSoup(html, "lxml")

In [22]:
for tag in soup.findAll('span', attrs={'class': re.compile('match-lineup-v2__player-name')}):
    print(tag.text)

 11. Miedema 
 25. Blackstenius 
 77. Heath 
 10. Little 
 12. Maanum 
 15. McCabe 
 7. Catley 
 3. Wubben Moy 
 6. Williamson 
 16. Maritz 
 1. Zinsberger 


## Eventos:

### Gol:
player_1 = quem fez o gol \
player_2 = assistência

### Cartão amarelo:
player_1 = quem recebeu o cartão amarelo \

### Penalti:
player_1 = quem fez o gol de penalti \
player_2 = "Penalti"

### Substituição:
player_1 = quem entra \
player_2 = quem sai

In [23]:
def get_events(list_of_elements):
    events_list = []
    for event in list_of_elements:
        event_type = event.find_element_by_tag_name('img').get_attribute('alt')
        event_text = event.text.split('\n')
        event_time = event_text[0]
        player_1 = event_text[1]
        if len(event_text) >= 3:
            player_2 = event_text[2]
        else:
            player_2 = ''
        event_detail = {
            'time': int(re.findall(r'\d+', event_time)[0]),
            'player_1': player_1,
            'player_2': player_2,
            'event': event_type
        }
        events_list.append(event_detail)
    return events_list

In [24]:
# Selecionar todos os eventos
search_box = driver.find_elements_by_xpath("//button[contains(@class, 'match-events__toggle-button')]")
# search_box
search_box[0].click()

C:\Users\user\AppData\Local\Temp\ipykernel_15652\3361420383.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@class, 'match-events__toggle-button')]")


In [25]:
# Listar todos os eventos da partida
search_box = driver.find_elements_by_xpath("//li[contains(@class, 'match-events__item')]")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\1877595458.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//li[contains(@class, 'match-events__item')]")


In [26]:
events = get_events(search_box)

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


In [27]:
pprint(events)

[{'event': 'Gol',
  'player_1': 'Jill Roord',
  'player_2': 'Sveindís Jónsdóttir',
  'time': 9},
 {'event': 'Substituição',
  'player_1': 'Alexandra Popp',
  'player_2': 'Lena Lattwein',
  'time': 48},
 {'event': 'Substituição',
  'player_1': 'Caitlin Foord',
  'player_2': 'Tobin Heath',
  'time': 46},
 {'event': 'Substituição',
  'player_1': 'Nikita Parris',
  'player_2': 'Katie McCabe',
  'time': 71},
 {'event': 'Substituição',
  'player_1': 'Jordan Nobbs',
  'player_2': 'Frida Maanum',
  'time': 71},
 {'event': 'Gol contra',
  'player_1': 'Leah Williamson',
  'player_2': 'Gol contra',
  'time': 72},
 {'event': 'Substituição',
  'player_1': 'Ewa Pajor',
  'player_2': 'Sveindís Jónsdóttir',
  'time': 79},
 {'event': 'Substituição',
  'player_1': 'Laura Wienroither',
  'player_2': 'Noelle Maritz',
  'time': 79},
 {'event': 'Substituição',
  'player_1': 'Rebecka Blomqvist',
  'player_2': 'Lena Oberdorf',
  'time': 86}]


## Estatísticas

In [28]:
search = driver.find_elements_by_xpath("//li[contains(@class, 'match-stats__list-item')]")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\2243698784.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//li[contains(@class, 'match-stats__list-item')]")


In [29]:
stats = get_stats(search)

In [30]:
pprint(stats)

[{'away_team_stat': '43%',
  'home_team_stat': '57%',
  'stat_name': 'POSSE DE BOLA'},
 {'away_team_stat': '8',
  'home_team_stat': '17',
  'stat_name': 'TOTAL DE CHUTES'},
 {'away_team_stat': '3', 'home_team_stat': '5', 'stat_name': 'CHUTES AO GOL'},
 {'away_team_stat': '45%',
  'home_team_stat': '55%',
  'stat_name': 'DISPUTAS GANHAS'}]


## Informações da partida

In [31]:
search = driver.find_elements_by_xpath("//li[contains(@class, 'match-info__entry')]")

C:\Users\user\AppData\Local\Temp\ipykernel_15652\1566266407.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//li[contains(@class, 'match-info__entry')]")


In [32]:
def get_infos(list_of_elements):
    infos_list = []
    for info in list_of_elements:
        info_text = info.text.split('\n')
        info_title = info_text[0]
        info_content = info_text[1]
        info_detail = {
            'info_title': info_title,
            'info_content': info_content,
        }
        infos_list.append(info_detail)
    return infos_list

In [33]:
infos = get_infos(search)

In [34]:
pprint(infos)

[{'info_content': 'Liga dos Campeões Feminina da UEFA',
  'info_title': 'Competição'},
 {'info_content': '31/03/22', 'info_title': 'Início'},
 {'info_content': 'VOLKSWAGEN ARENA', 'info_title': 'Estádio'}]


In [35]:
# driver.quit()